In [ ]:
'''
调用高德api/调用hmap接口（hmap联系孙修飞/陈善文），通过订单开始和结束经纬度，获取最短路径，对比订单里程，给出异常订单的推荐阈值
利用python线程池进行多线程调用Hmap-API，计算实际距离；

已经整理到/common/thread_pool_getURL.py
'''

In [1]:
import pandas as pd
target = 'ods_july_only_cut_cal_0701_0707_0826_1'

full_fd = pd.read_csv(f"C:\\Users\\zhengqiushi\\Desktop\\hik\\21-08-23\\data\\{target}.csv")
# full_fd['Hmap_dist'] >= -0.002 说明调用失败（筛去）
# full_fd['zke_lch_jyqk'] > 3    筛去公里数
full_fd_remained = full_fd[(full_fd['Hmap_dist'] <= 0) & (full_fd['Hmap_dist'] >= -0.002) & (full_fd['zke_lch_jyqk'] > 3)]

# 需要打包坐标对
all_start_Lon = list(full_fd_remained['cl_cfa_dqjd'].values)
all_start_Lat = list(full_fd_remained['cl_cfa_dqwd'].values)

all_end_Lon = list(full_fd_remained['cl_dd_dqjd'].values)
all_end_Lat = list(full_fd_remained['cl_dd_dqwd'].values)

all_start = list(zip(all_start_Lon, all_start_Lat))
all_end = list(zip(all_end_Lon, all_end_Lat))
all_dist = list(full_fd_remained['zke_lch_jyqk'].values)

_index = list(full_fd_remained.index.values)
    
big_group = list(zip(all_start, all_end, _index))

C:\Users\zhengqiushi\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
test_group = big_group

In [7]:
import requests
from multiprocessing.dummy import Pool as ThreadPool 
from multiprocessing import Process, Lock, Manager
import hmap_new_router # common/map-API/hmap_new_router
import time

# 并发请求数
max_worker = 5

# 累积成功数
success_so_far = 0
# 连续失败数
continue_fail_cnt = 0
# 是否终止
terminate_ = False
# 自动保存阈值
save_per_time = 100
# 自动终止阈值
terminate_fail_thresh = 400

# 失败集合
failed_set = []

# 线程处理函数
def get_status(per_group):
    global success_so_far
    global continue_fail_cnt
    global failed_set
    global terminate_
    global failed_set
    
    if terminate_ == True:
        return -1
    nowTime = time.strftime("%Y-%m-%d %H:%M:%S")
    # 调用hmap，请求距离与具体路线
    ret_route_Hmap, ideal_dist_Hmap = hmap_new_router.Hmap_new_ret_route_pack(per_group[0], per_group[1])

    if ideal_dist_Hmap > 0:
        # 添加成功情况
        success_lock.acquire() 
        success_so_far += 1 # 添加成功数
        full_fd.loc[per_group[2], ('Hmap_dist')] = ideal_dist_Hmap # 修改pandas内容
        success_lock.release()
        
        fail_lock.acquire()
        # 成功，连续失败数重新置零
        continue_fail_cnt = 0
        fail_lock.release()
        
        if success_so_far % save_per_time == save_per_time - 1:
            # 每成功save_per_time 条，自动保存一次
            cur_success = success_so_far
            file_lock.acquire()
            full_fd.to_csv(f"C:\\Users\\zhengqiushi\\Desktop\\hik\\21-08-23\\data\\{target}.csv",index=False)
            print(nowTime, "已完成: ", cur_success, "自动保存")
            file_lock.release()
    else :
        # 请求失败
        fail_lock.acquire()
        continue_fail_cnt += 1
        # 连续失败+1
        print(nowTime, per_group[2], "失败了，当前连续失败数：", continue_fail_cnt)
        failed_set.append(per_group)
        fail_lock.release()
        # 将Hmap_dist*2 ， 设置一般< -0.02 即失败两次就不再跳过了
        full_fd.loc[per_group[2], ('Hmap_dist')] += full_fd.loc[per_group[2], ('Hmap_dist')] # 修改！

        if continue_fail_cnt > terminate_fail_thresh:
            # 超过了失败阈值，直接退出
            print(nowTime, "超过失败阈值，准备退出", continue_fail_cnt)
            # 置终止！主进程看到后直接退出
            terminate_ = True

    return ideal_dist_Hmap

# 线程池初始化
def init(l_file, l_success, l_fail):
    global file_lock
    global success_lock
    global fail_lock
    file_lock = l_file
    success_lock = l_success
    fail_lock = l_fail

if __name__ == "__main__":
    # 线程池需要的全局变量
    file_lock = Lock()
    success_lock = Lock()
    fail_lock = Lock()

    all_group_num = len(test_group)
    # 计时
    t0 = time.time()

    # 定义线程池，并异步执行！
    pool = ThreadPool(max_worker, initializer=init, initargs=(file_lock, success_lock, fail_lock))  # Make the Pool of workers
    results = pool.map_async(get_status, test_group) #Open the urls in their own threads

    # 主进程检查！
    while(1): # 查看是不是已经终止
        if terminate_ == True:
            print("连续请求失败，已退出")
            pool.terminate()
            break
        
        if  results.ready() == True:# cur_len >= all_group_num:
            print("已经完成了所有的请求")
            break
        
        time.sleep(1)

    # 参考：https://zhuanlan.zhihu.com/p/22223656

    pool.close() #close the pool and wait for the work to finish 
    pool.join() 

    elapsed = time.time() - t0
    msg = '\n 总请求条数：{} ， 失败条数：{} , 总花费时间 {:.2f}s'
    print(msg.format(all_group_num, len(failed_set), elapsed))

    full_fd.to_csv(f"C:\\Users\\zhengqiushi\\Desktop\\hik\\21-08-23\\data\\{target}.csv",index=False)
    print("成功保存")

2021-09-01 13:57:53 224013 失败了，当前连续失败数： 1
2021-09-01 13:57:53 236983 失败了，当前连续失败数： 2
2021-09-01 13:57:53 248914 失败了，当前连续失败数： 3
2021-09-01 13:57:53 219328 失败了，当前连续失败数： 4
2021-09-01 13:57:54 224103 失败了，当前连续失败数： 5
2021-09-01 13:57:54 249257 失败了，当前连续失败数： 6
2021-09-01 13:57:54 219740 失败了，当前连续失败数： 7
2021-09-01 13:57:56 224391 失败了，当前连续失败数： 8
2021-09-01 13:57:56 224478 失败了，当前连续失败数： 9
2021-09-01 13:57:57 224956 失败了，当前连续失败数： 10
2021-09-01 13:57:58 225027 失败了，当前连续失败数： 11
2021-09-01 13:57:59 225576 失败了，当前连续失败数： 12
2021-09-01 13:57:59 226622 失败了，当前连续失败数： 13
2021-09-01 13:58:00 226710 失败了，当前连续失败数： 14
2021-09-01 13:58:01 228418 失败了，当前连续失败数： 15
2021-09-01 13:57:56 249496 失败了，当前连续失败数： 16
2021-09-01 13:58:02 249586 失败了，当前连续失败数： 17
2021-09-01 13:58:02 249600 失败了，当前连续失败数： 18
2021-09-01 13:58:03 251667 失败了，当前连续失败数： 19
2021-09-01 13:58:04 252243 失败了，当前连续失败数： 20
2021-09-01 13:58:04 252981 失败了，当前连续失败数： 21
2021-09-01 13:57:53 117294 失败了，当前连续失败数： 22
2021-09-01 13:57:54 238145 失败了，当前连续失败数： 23
2021-09-01 13:58:08 